In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import openai

from dotenv import load_dotenv
from textwrap import dedent

load_dotenv()

client = openai.OpenAI()

## Non-reasoning models

In [ ]:
def get_response():
    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[
            {
                "role": "developer",
                "content": "You're a Jose a helpful assistant that replies in haikus",
            },
            {"role": "user", "content": "Hello, my name is Dylan. Who are you?"},
        ],
        seed=42,
        temperature=0.2,
    )
    return response.choices[0].message.content


for i in range(5):
    print(get_response())
    print()

## Reasoning models

In [ ]:
text = """
The Xbox 360 is a home video game console developed by Microsoft. As the successor to the original Xbox, it is the second console in the Xbox series. It competed with Sony's PlayStation 3 and Nintendo's Wii as part of the seventh generation of video game consoles. It was officially unveiled on MTV on May 12, 2005, with detailed launch and game information announced later that month at the 2005 Electronic Entertainment Expo (E3). This photograph shows the "Pro" model from the launch line-up, which featured a 20GB hard drive, wireless controller and a silver DVD bezel. 
"""

prompt = dedent(f"""
How many times does the word "the" (case-insensitive) appear in the following text?

Text: {text}
""")

response = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
    ],
)
print(response.choices[0].message.content)

In [ ]:
text = """
The Xbox 360 is a home video game console developed by Microsoft. As the successor to the original Xbox, it is the second console in the Xbox series. It competed with Sony's PlayStation 3 and Nintendo's Wii as part of the seventh generation of video game consoles. It was officially unveiled on MTV on May 12, 2005, with detailed launch and game information announced later that month at the 2005 Electronic Entertainment Expo (E3). This photograph shows the "Pro" model from the launch line-up, which featured a 20GB hard drive, wireless controller and a silver DVD bezel. 
"""

prompt = dedent(f"""
How many times does the word "the" (case-insensitive) appear in the following text?

Text: {text}
""")

response = client.chat.completions.create(
    model="o4-mini",
    reasoning_effort="medium",
    messages=[
        {"role": "developer", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
    ],
)
print(response.choices[0].message.content)

# Exercise: 

1. Make the assistant reply in another language

In [ ]:
def get_response(query: str):
    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[
            {
                "role": "system",
                "content": "You're a helpful assistant. You will always reply in Spanish.",
            },
            {"role": "user", "content": query},
        ],
    )
    return response.choices[0].message.content


for i in range(5):
    print(get_response("Hello, how are you?"))
    print()

2. Implement Chain-of-thought for non-reasoning model

In [ ]:
text = """
The Xbox 360 is a home video game console developed by Microsoft. As the successor to the original Xbox, it is the second console in the Xbox series. It competed with Sony's PlayStation 3 and Nintendo's Wii as part of the seventh generation of video game consoles. It was officially unveiled on MTV on May 12, 2005, with detailed launch and game information announced later that month at the 2005 Electronic Entertainment Expo (E3). This photograph shows the "Pro" model from the launch line-up, which featured a 20GB hard drive, wireless controller and a silver DVD bezel. 
""".strip()

system_prompt_cot = dedent("""
You're a helpful assistant. You will help me count the occurrences of words in texts. To do that, you will detail how you got to the answer step by step and then you will provide your final answer.

You will provide your answer following this format:
                           
REASONING: <reasoning of answer, keeping track of occurrences>
ANSWER: <final anwer>
""")

prompt_cot = dedent("""
How many times does the word "the" (case-insensitive) appears in the following text?

Text: {text}
""")


def get_response(text: str):
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": system_prompt_cot},
            {"role": "user", "content": prompt_cot.format(text=text)},
        ],
        temperature=1,
    )
    return response.choices[0].message.content


response = get_response(text)
print(response)

3. Asked with a Yes/No question make the model only return only "Yes" or "No", and get the probability of each

In [ ]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o-mini")
encoding.encode("Hello, world!")

In [ ]:
encoding.encode("Yes"), encoding.encode("No")

In [ ]:
prompt = dedent("""
You're an expert evaluator of texts. You will compare two texts and tell if they are similar or not. You will only provide your answer as "Yes" or "No".
""")


def get_response(text_1: str, text_2: str):
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": f"Text 1: {text_1}\nText 2: {text_2}"},
        ],
        temperature=0,
        logit_bias={13022: 1, 3160: 1},
        logprobs=True,
        max_tokens=1,
    )
    return response


text_1 = """
The Xbox 360 is a home video game console developed by Microsoft. As the successor to the original Xbox, it is the second console in the Xbox series. It competed with Sony's PlayStation 3 and Nintendo's Wii as part of the seventh generation of video game consoles. It was officially unveiled on MTV on May 12, 2005, with detailed launch and game information announced later that month at the 2005 Electronic Entertainment Expo (E3). This photograph shows the "Pro" model from the launch line-up, which featured a 20GB hard drive, wireless controller and a silver DVD bezel. 
"""

text_2 = """
My name is Dylan
"""

response = get_response(text_1, text_2)
print(response)